This is the intuition behind [ReAct](https://react-lm.github.io/), a general agent architecture.
  
* `act` - let the model call specific tools 
* `observe` - pass the tool output back to the model 
* `reason` - let the model reason about the tool output to decide what to do next (e.g., call another tool or just respond directly)

This [general purpose architecture](https://blog.langchain.dev/planning-for-agents/) can be applied to many types of tools. 

![Screenshot 2024-08-21 at 12.45.43 PM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66dbac0b4a2c1e5e02f3e78b_agent2.png)

In [1]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model = "qwen2.5:7b")

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiply a and b.
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.
    """
    return a / b

tools = [add, multiply, divide]
llm_with_tools = llm.bind_tools(tools)#, parallel_tool_calls = False)

In [3]:
from langgraph.graph import StateGraph, MessagesState
from langchain_core.messages import HumanMessage, SystemMessage

system_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

def agent(state):
    return {"messages": [llm_with_tools.invoke([system_msg] + state["messages"])]}

We connect the `Tools` node *back* to the `Assistant`, forming a loop.

* After the `assistant` node executes, `tools_condition` checks if the model's output is a tool call.
* If it is a tool call, the flow is directed to the `tools` node.
* The `tools` node connects back to `assistant`.
* This loop continues as long as the model decides to call tools.
* If the model response is not a tool call, the flow is directed to END, terminating the process.

In [4]:
from langgraph.graph import START,END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image

builder = StateGraph(MessagesState)
builder.add_node("agent_node", agent)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "agent_node")
builder.add_conditional_edges("agent_node", tools_condition)
builder.add_edge("tools","agent_node")

graph = builder.compile()

# display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))
# Image(graph.get_graph().draw_mermaid_png())

In [5]:
messages = [HumanMessage(content="Add 3 and 4. Multiply the output by 2. Divide the output by 5")]
messages = graph.invoke({"messages": messages})

In [6]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Add 3 and 4. Multiply the output by 2. Divide the output by 5
================================== Ai Message ==================================
Tool Calls:
  add (23e821f7-d59c-49c7-a8f6-8f6b5aeebf53)
 Call ID: 23e821f7-d59c-49c7-a8f6-8f6b5aeebf53
  Args:
    a: 3
    b: 4
  multiply (0813737b-cd5f-4ef7-a684-9e36507c2fab)
 Call ID: 0813737b-cd5f-4ef7-a684-9e36507c2fab
  Args:
    a: 7
    b: 2
  divide (6824f487-99e8-4507-9a36-15eb66fb571f)
 Call ID: 6824f487-99e8-4507-9a36-15eb66fb571f
  Args:
    a: 14
    b: 5
================================= Tool Message =================================
Name: add

7
================================= Tool Message =================================
Name: multiply

14
================================= Tool Message =================================
Name: divide

2.8
================================== Ai Message ==================================

The final result after ad